# Multi-layer Forward-feed NN 

## 0. Finish Facial Recognition from last time

## 1. Learn to pickle a dictionary

## 2. Alak -- Checker and Chess Meet with Go (棋)
- ### Explain the rules
- ### Simulate a game

In [1]:
%matplotlib inline
# All imports

from __future__ import print_function, division
from random import choice
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
import time
from IPython import display

import sklearn
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.cross_validation import train_test_split

from time import time
from sklearn.metrics import classification_report, confusion_matrix

# -------------> New import <------------------

import pickle

np.set_printoptions(formatter={'float': '{:.5f}'.format})


/Users/labuser/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
test_dict = {'african elephant': 3, 'blue whale': 30}
test_dict['polar bear'] = 2.5
print(test_dict)

pickle.dump(test_dict, open('test_pickle.p', 'w'))

{'blue whale': 30, 'polar bear': 2.5, 'african elephant': 3}


In [3]:
# Best way

with open('test_pickle.p', 'w') as f:
    pickle.dump(test_dict, f)

In [4]:
# To open
read_dict = pickle.load(open('test_pickle.p', 'r'))
print(read_dict)

{'blue whale': 30, 'polar bear': 2.5, 'african elephant': 3}


## Minibreakout: Write the above example using the 

      with...
      
## syntax.  Call the dictionary object read from pickle, read_dict2.

In [5]:
with open('test_pickle.p', 'r') as f:
    read_dict2 = pickle.load(f)
print (read_dict2)

{'blue whale': 30, 'polar bear': 2.5, 'african elephant': 3}


## Simulate Alak:
   ### a) Randomly generate the next legal move
   ### b) Decide if any pieces need to be removed from the board
   ### c) The opposite side do step a), then b), etc.
   ### d) Repeat until one side wins (the other side should have zero piece on the board)
   ### Save all the steps of the simulated game in a pickle file (save one side as 'x', the other side as 'o', and unoccupied as '\_' -- thus each step will be saved as a string.  The key should be the step number)


In [7]:
from pdb import set_trace
import regex as re

# class Alak:
#     def __init__(self):
#         self.board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])
#         self.x_pos = np.where(self.board == 'x')[0]
#         self.o_pos = np.where(self.board == 'o')[0]
#         self.random_move = np.random.choice(np.where(self.board == '_')[0])
#         self.step = []
        
#     def next_move(self):
#         self.random_move = np.random.choice(np.where(self.board == '_')[0])
        
    
def next_move(board, piece):
    move = np.random.choice(np.where(board == '_')[0])
    
    piece_pos = np.where(board == str(piece))[0]    #Indices of x/o pieces
    rand_piece = np.random.choice(piece_pos)     #Among pieces, choose random piece to move to new position, move
    
    board[move], board[rand_piece] = board[rand_piece], board[move]     #Assign piece to move
    
    step = "{}: {} --> {}".format(piece, rand_piece, move)     #Assigns step of piece to new position
    return board, step

def check_dead(board, piece):
    if piece == 'x':
        dead_pat = re.search('xo+x', ''.join(board))
        if dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board), overlapped=True)]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
        dead_pat = re.search('ox+o', ''.join(board))
        if dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board), overlapped=True)]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            return board
        else:
            return board
    else:
        dead_pat = re.search('ox+o', ''.join(board))
        if dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board), overlapped=True)]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
        dead_pat = re.search('xo+x', ''.join(board))
        if dead_pat != None:
            dead_se = [(m.start(0), m.end(0)) for m in re.finditer(dead_pat.group(), ''.join(board), overlapped=True)]
            for elem in dead_se:
                np.put(board, range(elem[0]+1, elem[1]-1), '_')
            return board
        else:
            return board

steps = []
board = np.array(['x', 'x', 'x', 'x', '_', '_', 'o', 'o', 'o', 'o'])
print(*board)
print()
# board = np.array(['x', '_', 'o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
i = 1


# while 'x' in board and 'o' in board:
while len(np.unique(board)) > 2:
    print("Round: {}\n".format(i))
############################################
    if len(np.unique(board)) > 2:
        board, step = next_move(board, 'x')     #Determines which piece to move
        steps.append(step)
        print(step)
        print(*board)
        board_up = check_dead(board, 'x')
        if board_up != None:
            board = board_up
        print(*board)
        print()
############################################
    if len(np.unique(board)) > 2:
        board, step = next_move(board, 'o')
        steps.append(step)
        print(step)
        print(*board)
        board_up = check_dead(board, 'o')
        if board_up != None:
            board = board_up
        print(*board)
        print()
    
    i += 1

x x x x _ _ o o o o

Round: 1

x: 0 --> 5
_ x x x _ x o o o o
_ x x x _ x o o o o

o: 6 --> 4
_ x x x o x _ o o o
_ x x x _ x _ o o o

Round: 2

x: 2 --> 6
_ x _ x _ x x o o o
_ x _ x _ x x o o o

o: 9 --> 4
_ x _ x o x x o o _
_ x _ x o _ _ o o _

Round: 3

x: 3 --> 6
_ x _ _ o _ x o o _
_ x _ _ o _ x o o _

o: 8 --> 9
_ x _ _ o _ x o _ o
_ x _ _ o _ x o _ o

Round: 4

x: 1 --> 3
_ _ _ x o _ x o _ o
_ _ _ x o _ x o _ o

o: 7 --> 8
_ _ _ x o _ x _ o o
_ _ _ x o _ x _ o o

Round: 5

x: 6 --> 2
_ _ x x o _ _ _ o o
_ _ x x o _ _ _ o o

o: 8 --> 7
_ _ x x o _ _ o _ o
_ _ x x o _ _ o _ o

Round: 6

x: 2 --> 6
_ _ _ x o _ x o _ o
_ _ _ x o _ x o _ o

o: 4 --> 2
_ _ o x _ _ x o _ o
_ _ o x _ _ x o _ o

Round: 7

x: 6 --> 8
_ _ o x _ _ _ o x o
_ _ o x _ _ _ o _ o

o: 7 --> 8
_ _ o x _ _ _ _ o o
_ _ o x _ _ _ _ o o

Round: 8

x: 3 --> 1
_ x o _ _ _ _ _ o o
_ x o _ _ _ _ _ o o

o: 2 --> 3
_ x _ o _ _ _ _ o o
_ x _ o _ _ _ _ o o

Round: 9

x: 1 --> 6
_ _ _ o _ _ x _ o o
_ _ _ o _ _ x _ o o

o: 3 

/Users/labuser/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:75: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/Users/labuser/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:86: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


# End of Week11-1

In [25]:
test_board = np.array(['o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
stuff = re.search('ox+o', ''.join(test_board))
new_test_board = ''.join(test_board)
# print(np.where(new_test_board == stuff.group().rsplit()))
print(new_test_board)
print(stuff.group())

oxoxoo__
oxo


In [189]:
test_board = np.array(['x', 'x', '_', 'x', 'o', 'o', 'x', '_', '_', 'o'])
test_board_j = ''.join(test_board)
stuff = re.search('xox', ''.join(test_board))
if stuff != None:
    stuff = stuff.group()
    se = [(m.start(0), m.end(0)) for m in re.finditer(stuff, test_board_j)]
    for elem in se:
        print(test_board)
        np.put(test_board, range(elem[0]+1, elem[1]-1), '_')
        print(test_board)
        
    
# stuff_new = np.array(list(stuff.group()))
# print(np.where(test_board == stuff_new))

TypeError: first argument must be string or compiled pattern

In [13]:
import regex as re

pattern = 'oxo'
string = 'oxoxxo'
se = [(m.start(0), m.end(0)) for m in re.finditer(pattern, string, overlapped=True)]
print(se)
# for elem in se:
    

[(0, 3)]


In [79]:
stuff = np.array(['x', 'o', 'x', '_'])
print(np.unique(stuff))
cool = np.array(['_', 'o', 'x'])
print(np.unique(stuff) == cool)

['_' 'o' 'x']
[ True  True  True]


In [7]:
new_board = np.array(['x', '_', 'o', 'x', 'o', 'x', 'o', 'o', '_', '_'])
print(check_dead(new_board, 'o'))

['x' '_' 'o' '_' 'o' 'x' 'o' 'o' '_' '_']


In [8]:
stuff_board = np.array(['o', 'x', 'o', 'x', 'x', 'o', '_', '_'])
print(check_dead(stuff_board, 'o'))

['o' '_' 'o' 'x' 'x' 'o' '_' '_']
